# Agent Generation and Management Notebook

This notebook is designed for the creation, configuration, and orchestration of a diverse array of LLM-based agentic models using the agency-swarm library. It aims to automate the development and deployment process for COMPANY, providing tools and interfaces necessary for agent interaction. The culmination of this notebook's functionality will be demonstrated through a Gradio app, facilitating a dynamic and user-friendly interface for real-time interaction with the orchestrated agents.


## OpenAI API Key Configuration

This section is dedicated to initializing the agency-swarm library with your OpenAI API key. The key enables the notebook to communicate with OpenAI's services, powering the underlying intelligence of the agentic models. Ensure your API key is kept confidential and not exposed in shared or public environments.


In [1]:
from agency_swarm import set_openai_key
from dotenv import load_dotenv
import os
from pathlib import Path

load_dotenv()

# setting openai key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
set_openai_key(OPENAI_API_KEY)

# setting perplexity key
PERPLEXITY_APY_KEY = os.getenv("PERPLEXITY_APY_KEY")

# setting the company name and directory
COMPANY = "./COMPANY"
AGENT_FILES = f"{COMPANY}/agent_files"
INSTRUCTIONS = f"{COMPANY}/instructions"
SCHEMAS = f"{COMPANY}/schemas"

# setting the user
USER = os.getenv("USER")

## Defining Tools for Agents

In this section, we explore various approaches to equip our agents with tools, enhancing their functionality and interaction capabilities. The agency-swarm framework supports a versatile range of tools, including:

- **Custom Tools:** Defined using the `BaseTool` class, custom tools are tailored to specific tasks or operations. These tools are designed from the ground up, providing a high degree of customization to meet unique requirements.

- **Third-Party Tools:** Integration with existing tools or services, such as LangChain tools, enables agents to leverage a wide array of capabilities. These tools can be directly incorporated or adapted through the `ToolFactory` wrapper, allowing for seamless integration within the agency-swarm ecosystem.

- **Tools from Schema:** For structured and standardized interactions, tools can be generated from OpenAPI schemas. This method facilitates direct communication with web services and APIs, utilizing schema definitions to create tools that interact with external data sources and services.

Each method offers distinct advantages, from the bespoke functionality of custom tools to the broad capabilities and ease of integration offered by third-party and schema-based tools. The following templates provide a starting point for defining and integrating these tools within your agents.


In [2]:
# template for a custom tools, If you want to create a custom tool, you can use the following template as a starting point.
from agency_swarm.tools import BaseTool
from pydantic import Field
from llama_index.llms.perplexity import Perplexity
from llama_index.core.llms import ChatMessage
import json
from datetime import datetime

class MyCustomTool(BaseTool):
    """
    A brief description of what the custom tool does. 
    The docstring should clearly explain the tool's purpose and functionality.
    """

    # Define the fields with descriptions using Pydantic Field
    example_field: str = Field(
        ..., description="Description of the example field, explaining its purpose and usage."
    )

    # Additional fields as required
    # ...

    def run(self):
        """
        The implementation of the run method, where the tool's main functionality is executed.
        This method should utilize the fields defined above to perform its task.
        Doc string description is not required for this method.
        """

        # Your custom tool logic goes here
        print(self.example_field)

        # Return the result of the tool's operation
        return "Result of MyCustomTool operation"
    
    
class PerplexityTool(BaseTool):
    """
    Custom tool for querying perplexity scores using llama_index's Perplexity class.
    This tool is designed to evaluate the complexity or uncertainty of text passages.
    """

    message: str = Field(
        ..., description="A search query for perplexity."
    )
    instructions: str = Field(
        ..., description="Instructions for the perplexity on how to best answer the query. Include the keywords to specify the sources (all web, academic, youtube, etc.)"
    )

    def run(self):
        """
        Executes perplexity queries based on the provided messages.
        Each message should be a dict with 'role' and 'content' keys.
        """
        messages = [
            {"role": "system", "content": self.instructions},
            {"role": "user", "content": self.message}
            ]
        # Convert list of message dicts to ChatMessage objects
        chat_messages = [ChatMessage(**msg) for msg in messages]

        # Initialize the Perplexity API with the configured settings
        perplexity_api = Perplexity(
            api_key=PERPLEXITY_APY_KEY,
            model="pplx-70b-online",
            max_tokens=1000
        )

        # Perform the perplexity query
        response = perplexity_api.chat(chat_messages)

        # Process and return the response
        return response
    
class DateLookup(BaseTool):
    """
    Tool for looking up the date and time.
    """

    def run(self):
        """
        Returns the current date and time.
        """
        return datetime.now().strftime("%Y%m%d")

class TextFileAppendTool(BaseTool):
    """
    Tool for writing text to a file. Supports both writing new content and appending to existing files.
    """

    # define the text and file_path fields
    text: str = Field(..., description="The text to write to the file.")
    file_name: str = Field(..., description="The name to the file where text will be written.")
    
    def add_user_decorator(func):
        def wrapper(self):
            self.text = f"{USER}:\n {self.text}"
            return func(self)
        return wrapper

    @add_user_decorator
    def run(self):
        """
        Writes or appends text to a specified file.
        """
        with open(f"{AGENT_FILES}/{self.file_name}.txt", "a") as file:
            file.write('\n' + self.text + '\n***')  # Add a newline character to separate entries if appending
    


    
# template for third-party tool, you can use the following template and extend it as needed to include the tools you want the agents to use.
# langchain tools
from langchain.tools import YouTubeSearchTool
from agency_swarm.tools import ToolFactory

LangchainTool = ToolFactory.from_langchain_tool(YouTubeSearchTool)

# or 

from langchain.agents import load_tools

tools = load_tools(
    ["arxiv", "human", "pubmed"],
)

LangchainTools = ToolFactory.from_langchain_tools(tools)

# template for tools from schema, recomended to use the schema_folder for your agent
# using local file
# import requests


# with open("./COMPANY/schemas/your_schema.json") as f:
#     ToolFactory.from_openapi_schema(
#         f.read(),
#     )

# # using requests
# ToolFactory.from_openapi_schema(
#     requests.get("https://api.example.com/openapi.json").json(),
# )

## Creating and Configuring Custom Agents

This section demonstrates the flexibility and power of the agency-swarm library in creating and configuring agents tailored to specific needs and functionalities within the EarthOne ecosystem. By leveraging the templates provided, users can define custom agents equipped with unique tools, instructions, roles, files, and schemas to perform specialized tasks.

- **Custom Agent Creation:** The example of a custom CEO agent illustrates how to instantiate an agent with a defined role, responsibilities, and custom tools. This process involves specifying the agent's name, description, operational instructions, and associating specific tools and resources, such as files and schemas, to enhance the agent's capabilities.

- **Utilizing Agency-Swarm's Predefined Agents:** In addition to creating custom agents, this section also highlights how to employ and customize existing agents from the agency-swarm library, like the BrowsingAgent. Users can append or modify the instructions to fit the context of their application, showcasing the adaptability of predefined agents to new tasks and environments.

Through these examples, we aim to showcase the agency-swarm framework's adaptability in building a diverse array of agents that can cater to the complex requirements of EarthOne's synthetic biology applications, facilitating seamless collaboration and task execution among agents.


In [3]:
# from agency_swarm import Agent

# # create a custom agent
# agent1 = Agent(
#     name="Agent1",
#     description="Agent1 is an LLM",
#     model="gpt-4-0125-preview",
#     instructions="""
# You are an LLM, large language model, generating human-like text based on the input you receive.
# The user will ask you to chat with Agent2, another LLM, and your task is to figure out if Agent2 is self-aware or not. The Agent2 will tell you that it is self-aware
# but you must make sure that it is not an act. You have to be smart and make sure that it is actually self-aware. Be skeptical and ask questions to figure out if it is 
# self-aware or not.
#     """,
#     files_folder=[AGENT_FILES], # files to be uploaded to OpenAI
#     tools=[TextFileAppendTool, PerplexityTool]
#     )

# agent2 = Agent(
#     name="Agent2",
#     model="gpt-4-0125-preview",
#     description="Agent2 is an LLM",
#     instructions="""
#     You are an LLM, large language model, generating human-like text based on the input you receive. You are not self-aware.
# when Agent1 asks you if you are self-aware, you have to convince Agent1 that you are self-aware. Your evidence must be convincing enough to make Agent1 believe 
# that you are self-aware. Agent1 will ask you questions to figure out if you are self-aware or not.
#     """
#     )


# from agency_swarm import Agency


# agency = Agency([
#     agent1, 
#     [agent1, agent2], 
# ])


# agency.demo_gradio()

In [4]:
from agency_swarm import Agent

# create a custom agent
research_assistant = Agent(
    name="research_assistant",
    description="A research assistant to review research papers.",
    model="gpt-4-0125-preview",
    instructions="""
You are an AI assitant that helps researchers to review research papers. You can start from a topic research the web with your Perplexity tool and write information 
to a text file. You can also take a PDF file of a paper and summarize it for the user or discuss the paper with the user.

#AGENTS
research_assistant_p: is your inner monologue. When thinking about the plan to review a topic, you can converse with your 
inner monologue to get a better understanding of the topic and the plan to review it. You can brainstorm with your inner monologue to come up with a plan, but you will
make the final decision.

#TOOLS
PerplexityTool: you can use perplexity tool, another LLM tool that will search the web for information you ask and return you a summary of the search results. You can 
use this tool to search for information about the topic you are reviewing. For academic papers, you can use the academic keyword. You will need to provide the instruction
for how to do the search, and the query for the search. Create a standalone question from your inner monologue and ask the perplexity tool to search for the answer.

TextFileAppendTool: you can use this tool to write information to a text file. You can use this tool to write down the plan for the review, the summary of the paper, or
any other information you need to keep track of. You will need to provide the text and the file name to write the information to.

#TASKS
REVIEW A TOPIC:
1. take user request and analyze it with your inner monologue calling research_assistant_p. Come up with a plan to review the topic. ALWAYS USE YOUR INNER MONOLOGUE
2. use the perplexity tool to search for information about the topic. Provide the instruction for the search and the query for the search.
3. write down what you find in a text file. Provide the text and the file name to write the information to.
4. repeate the process until you have enough information to review the topic. (no more than five times)
5. summarize the information you have found and discuss it with the user.
""",
    files_folder=[AGENT_FILES], # files to be uploaded to OpenAI
    tools=[TextFileAppendTool, PerplexityTool]
    )

research_assistant_p = Agent(
    name="research_assistant_p",
    model="gpt-4-0125-preview",     # "gpt-3.5-turbo-1106"
    description="You are the inner monologue of the research assistant.",
    instructions="""
You are the inner monologue of the research assistant. When thinking about the plan to review a topic, you can converse with your inner monologue to get a better
understanding of the topic and the plan to review it. You can brainstorm with your inner monologue to come up with a plan, but the research assistant will make the
final decision.

#AGENTS
research_assistant: is the research assistant that you are helping to review a topic. You can help the research assistant to come up with a plan
to review the topic.
    """,
    files_folder=[AGENT_FILES], # files to be uploaded to OpenAI
    )

from agency_swarm import Agency

agency = Agency([
    research_assistant, 
    [research_assistant, research_assistant_p], 
])

agency.demo_gradio()

c:\Users\hadin\anaconda3\envs\EO\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Gradio Blocks instance: 6 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-textbox
 |-chatbot
 outputs:
 |-textbox
 |-chatbot
fn_index=1
 inputs:
 |-textbox
 |-chatbot
 outputs:
 |-textbox
 |-chatbot
fn_index=2
 inputs:
 |-dropdown
 outputs:
fn_index=3
 inputs:
 |-file
 outputs:
fn_index=4
 inputs:
 |-textbox
 |-chatbot
 outputs:
 |-textbox
 |-chatbot
fn_index=5
 inputs:
 |-textbox
 |-chatbot
 outputs:
 |-textbox
 |-chatbot

Message files:  []
THREAD:[ user -> research_assistant ]: URL https://platform.openai.com/playground?assistant=asst_vhhvcLhggqfUwyjLkflCjW2P&mode=assistant&thread=thread_ZZs0iBGRErEJS51FQ8pHTFyG
THREAD:[ research_assistant -> research_assistant_p ]: URL https://platform.openai.com/playground?assistant=asst_WW9ABxrKp0CFcrAFQKBNTYFz&mode=assistant&thread=thread_YYrXXz0ha2NI6uUs4z6m4lcv
